#This File is used to generate sp_500_Stocks.csv file


- Download sp_500_stocks.csv from Kaggle
- Upload to your google drive 'My Drive' folder
- Run this code and it will download the version of the sp_500_stocks.csv we use in D3


In [139]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import StandardScaler
import datetime as dt
from pandas.tseries.offsets import Day
from datetime import datetime

In [140]:
drive.mount('/content/drive')
sp500_stocks = pd.read_csv('/content/drive/My Drive/sp500_stocks.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [141]:
sp500_stocks = sp500_stocks[['Date', 'Symbol', 'Adj Close']]
sp500_stocks

,Date,Symbol,Adj Close
0,2010-01-04,MMM,59.318886
1,2010-01-05,MMM,58.947342
2,2010-01-06,MMM,59.783295
3,2010-01-07,MMM,59.826176
4,2010-01-08,MMM,60.247749
...,...,...,...
1797717,2024-03-11,ZTS,183.490005
1797718,2024-03-12,ZTS,181.350006
1797719,2024-03-13,ZTS,176.229996
1797720,2024-03-14,ZTS,173.880005


In [150]:
def reshape_data_to_rolling_returns_renamed(df):
    # Ensure the 'Date' column is of datetime type
    df['Date'] = pd.to_datetime(df['Date'])

    # Sort the dataframe by symbol and then date to ensure the pct_change is calculated correctly
    df = df.sort_values(by=['Symbol', 'Date'])

    # Calculate daily returns as percentage change of the adjusted close prices
    df['Daily Return'] = df.groupby('Symbol')['Adj Close'].pct_change()

    # Pivot the table to get Dates as columns and Symbols as rows with daily returns as values
    pivot_df = df.pivot(index='Symbol', columns='Date', values='Daily Return')

    # Convert the MultiIndex columns (levels) to a Index of Timestamps for proper filtering
    pivot_df.columns = pivot_df.columns.get_level_values('Date')

    # Get the most recent date to identify the end of the rolling year range
    most_recent_date = pivot_df.columns.max().date()

    # Calculate rolling yearly returns up to the present day, with "Year 1" being the oldest
    for i in range(5, 0, -1):
        # Define the start and end dates for the rolling year
        end_date = most_recent_date - pd.Timedelta(days=365 * (i - 1))
        start_date = most_recent_date - pd.Timedelta(days=365 * i)

        # Filter the dates for the year we are calculating and ensure they are within the DataFrame
        valid_dates = [date for date in pivot_df.columns if isinstance(date, pd.Timestamp) and start_date <= date.date() <= end_date]

        # Calculate the product of (1 + daily returns) and subtract 1 to get the cumulative return
        yearly_returns = (pivot_df[valid_dates] + 1).prod(min_count=1, axis=1) - 1

        # Assign yearly returns to the corresponding new column in the pivot table
        # Year naming is reversed here: 5 - i + 1
        pivot_df[f'Year {5 - i + 1} Returns'] = yearly_returns

    # Remove the time part from the DateTime index to match the input format
    pivot_df.columns = [col.date() if isinstance(col, pd.Timestamp) else col for col in pivot_df.columns]

    return pivot_df



In [151]:
d = reshape_data_to_rolling_returns(sp500_stocks)
d

,2010-01-04,2010-01-05,2010-01-06,2010-01-07,2010-01-08,2010-01-11,2010-01-12,2010-01-13,2010-01-14,2010-01-15,...,2024-03-11,2024-03-12,2024-03-13,2024-03-14,2024-03-15,Year 5 Returns,Year 4 Returns,Year 3 Returns,Year 2 Returns,Year 1 Returns
Symbol,,,,,,,,,,,,,,,,,,,,,
A,NaN,-0.010862,-0.003554,-0.001296,-0.000324,0.000649,-0.012005,0.007882,0.014988,-0.023114,...,-0.003922,0.003734,-0.000135,-0.017657,0.015632,0.100351,0.050200,0.094246,0.791876,-0.202498
AAL,NaN,0.113208,-0.041431,0.029470,-0.019084,-0.019455,0.007937,0.078740,0.020073,-0.016100,...,-0.002044,-0.047099,0.002865,-0.000714,-0.002144,0.007215,-0.093128,-0.345650,0.709993,-0.486687
AAPL,NaN,0.001729,-0.015906,-0.001849,0.006649,-0.008821,-0.011375,0.014106,-0.005791,-0.016712,...,0.011832,0.002779,-0.012123,0.010927,-0.002197,0.134331,0.010988,0.294770,0.820964,0.318399
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.004361,0.007181,-0.005859,0.007450,-0.018322,0.154615,-0.003721,0.481242,0.367771,-0.019890
ABNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.011643,0.022578,-0.011460,0.010197,-0.034847,0.414333,-0.166447,-0.258298,0.382143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,-0.003420,-0.007149,-0.000288,0.000288,0.017281,0.018969,0.003075,0.010866,-0.016814,...,0.003726,0.004712,-0.016911,-0.008673,-0.002333,0.097070,0.138390,0.125435,0.413815,-0.296374
ZBH,NaN,0.031656,-0.000323,0.022940,-0.021004,0.022100,-0.019255,0.006598,0.002878,-0.015622,...,0.010257,0.001328,-0.012636,-0.006794,-0.004454,0.007036,0.061040,-0.216791,0.651610,-0.263997
ZBRA,NaN,-0.001744,-0.007687,-0.025000,-0.003250,0.003261,-0.001083,0.014100,0.002852,-0.007821,...,-0.008351,0.018592,-0.003293,-0.009596,0.014515,-0.006534,-0.238064,-0.159921,1.575263,-0.214380


In [152]:
d.to_csv('sp_500_stocks.csv', index=True)